**Autogen Library Onboarding & Book Recommendation System**

This showcases an interactive system designed to onboard users and recommend books based on their preferences. It utilizes the Autogen framework to create multiple agents that collaborate to gather user information, recommend books, and execute necessary tasks such as file handling. Below is a brief overview of the key components and topics involved in the system:

**Key Components:**
Conversable Agents:
 
Login Agent: This agent helps onboard users by gathering basic information like their name and ID. The interaction is simple, and the agent terminates once the information is collected.
Interest Agent: After the login information is gathered, this agent asks users for their reading preferences, such as preferred genres.
Books Recommender Agent: Based on the user's genre preference, this agent recommends a list of books that align with their interest.
Customer Proxy Agent: This acts as a middleman between the user and the other agents, facilitating communication and information exchange.
Interaction Flow:


**Code Execution Agents:**

Code Executor Agent: This agent is responsible for executing commands such as file creation and data storage. It uses a LocalCommandLineCodeExecutor to run necessary scripts and handle data processing tasks.
Code Writer Agent: Once the user information is gathered, the code writer agent ensures that the data is appended to a library.csv file, creating the file if it doesn’t already exist. This ensures that all user data is saved for future reference and analysis.


**Tools/Framework used**
- Autogen ConversableAgent 
- Autogen AssistantAgent
- AzureOpenAi model used in backend
- LocalCommandLineCodeExecutor


In [245]:
import autogen
from autogen import ConversableAgent

In [ ]:
#using azureopenai model
llm_config={
        "model": "gpt-4",
        "api_key": "your api key",
        "api_type": "azure",
        "base_url": "your url for the model",
        "api_version":"2024-05-01-preview"}

In [ ]:
#define conversable agents
login_agent = ConversableAgent(
    name="login_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help every user to  get started with our library.
    Your job is to gather customer's name and id.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [248]:
interest_agent = ConversableAgent(
    name="interest_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help customers get started with our libray
    Your job is to gather customer's preferences for books to read like genre they would want to read.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [249]:
books_recommender_agent = ConversableAgent(
    name="books_recommender_agent",
    system_message='''You are a helpful customer service agent
    here to provide books suggestions based on the preference they had shared. Suggest upto to 5 top suggestions of books.Recommend books the requested genre ensuring all recommendations are appropriate for general audiences."
    Please ensure the response adheres to OpenAI's content policies and avoids sensitive topics
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [250]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [251]:
chats = [
    {
        "sender": login_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our library."
            "Could you tell me your name and id?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'id': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": interest_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
         "summary_args": {
        "summary_prompt": "Return the genre as a JSON object only: {'genre': ''}",
        },
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": books_recommender_agent,
        "message": "Let's find something fun to read in the {genre} genre.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as a json data object which has username, userid, genre, recommended books without markdown line formatting. character.user format {username:value,userid:value,genre:value,booksrecommended:[value,value,value]} user actual values"
        },
    },
]

In [252]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
login_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our library.Could you tell me your name and id?

--------------------------------------------------------------------------------


c:\Users\deej\Documents\MLworks\AI\venv\Lib\site-packages\autogen\agentchat\chat.py:55: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to login_agent):

harrison 3477

--------------------------------------------------------------------------------
login_agent (to customer_proxy_agent):

Thank you, Harrison. I have your ID as 3477.  
TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

********************************************************************************
Starting a new chat....

********************************************************************************
interest_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{
  "name": "Harrison",
  "id": "3477"
}

--------------------------------------------------------------------------------
customer_proxy_agent (to interest_agent):

comics

--------------------------------------------------------------------------------

*******************************************************************************

In [253]:
chat_results[2].summary
result=chat_results[2].summary

In [254]:
#the chat result summary has markdown characters hence using re module to clean and then create a dictionary for thr next steps
print(repr(result.strip()))

'{\n  "username": "Harrison",\n  "userid": "3477",\n  "genre": "comics",\n  "booksrecommended": [\n    "Bone by Jeff Smith",\n    "Ms. Marvel: No Normal by G. Willow Wilson and Adrian Alphona",\n    "Lumberjanes Series by Noelle Stevenson, Grace Ellis, Shannon Watters, and Brooke Allen",\n    "Amulet Series by Kazu Kibuishi",\n    "Batman: The Long Halloween by Jeph Loeb and Tim Sale"\n  ]\n}'


In [255]:
import re
clean_result = re.sub(r"```(json)?\n?", "", result).strip("`")
import json
data= json.loads(clean_result)

In [256]:
from autogen.coding import LocalCommandLineCodeExecutor
executor = LocalCommandLineCodeExecutor(
    timeout=60,
    work_dir="librarycodes",
)


In [257]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

In [258]:
from autogen import AssistantAgent
code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [259]:
code_writer_agent_system_message = code_writer_agent.system_message

In [ ]:
#let us write the date to the csv file
import datetime

today = datetime.datetime.now().date()
message = f"Make sure 'library.csv' exists. If it doesn't, create it. Then, store the following user information: {data} with today's date ({today})."


In [261]:
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message,
)

code_executor_agent (to code_writer_agent):

Make sure 'library.csv' exists. If it doesn't, create it. Then, store the following user information: {'username': 'Harrison', 'userid': '3477', 'genre': 'comics', 'booksrecommended': ['Bone by Jeff Smith', 'Ms. Marvel: No Normal by G. Willow Wilson and Adrian Alphona', 'Lumberjanes Series by Noelle Stevenson, Grace Ellis, Shannon Watters, and Brooke Allen', 'Amulet Series by Kazu Kibuishi', 'Batman: The Long Halloween by Jeph Loeb and Tim Sale']} with today's date (2025-02-17).

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

To achieve this, I will write a Python script that checks for the existence of 'library.csv'. If the file does not exist, it will create it and add a header row. The script will then append the user's information (including today's date) to the file.

```python
# filename: manage_library.py
import os
import csv
from datetime import date

# De